# Assignment 2 2AMM10 2023-2024

## Group: [Shash_Kas_Pim]
### Member 1: [Shashank Prabhu]
### Member 2: [Kasra Gheytuli]
### Member 3: [Pim de Wildt]

We need to install some specific libraries. The cell below installs torch_geometric for torch 2.6.0+cu124. In case the current version of torch is different, check [here](https://pytorch-geometric.readthedocs.io/en/latest/install/installation.html) to see which versions (of both libraries) you should install. You might also need to install an old version of torch from [here](https://pytorch.org/get-started/previous-versions/)

**Note:** Do not install pyg_lib from the optional dependencies

In [ ]:
!pip show torch

Name: torch
Version: 2.7.0+cu128
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: C:\Users\shash\OneDrive - TU Eindhoven\Shashank Prabhu University\Masters DSAI\Q4\2AMM10 Deep Learning\2AMM10-Assignments\venv\Lib\site-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, typing-extensions
Required-by: torchaudio, torchvision


In [ ]:
# !pip install rdkit
# !pip install torch_geometric
# !pip install torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.6.0+cu124.html

   ---------------------------------------- 0.0/22.4 MB ? eta -:--:--
   ------ --------------------------------- 3.7/22.4 MB 27.3 MB/s eta 0:00:01
   ------------------ --------------------- 10.5/22.4 MB 29.8 MB/s eta 0:00:01
   ----------------------------- ---------- 16.8/22.4 MB 30.2 MB/s eta 0:00:01
   ---------------------------------------  22.3/22.4 MB 30.7 MB/s eta 0:00:01
   ---------------------------------------- 22.4/22.4 MB 29.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 6.2 MB/s eta 0:00:00

   ----- ---------------------------------- 1/8 [multidict]
   -------------------- ------------------- 4/8 [yarl]
   ------------------------- -------------- 5/8 [aiosignal]
   ------------------------------ --------- 6/8 [aiohttp]
   ------------------------------ --------- 6/8 [aiohttp]
   ------------------------------ --------- 6/8 [aiohttp]
   ------------------------------ --------- 6/8 [aiohttp]
   ------------------------------ --------- 6/8 [aiohttp]
   ------------------------------ --------- 6/8 [aiohttp]
   ------------------------------ --------- 6/8 [aiohttp]
   ------------------------------ --------- 6/8 [aiohttp]
   ----------------------------------- ---- 7/8 [torch_geometric]
   ----------------------------------- ---- 7/8 [torch_geometric]
   ----------------------------------- ---- 7/8 [torch_geometric]
   --------------


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
     ---------------------------------------- 3.5/3.5 MB 20.6 MB/s eta 0:00:00
     ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
     ---------------------------------------- 2.1/2.1 MB 23.3 MB/s eta 0:00:00
     ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
     ---------------------------------------- 1.6/1.6 MB 29.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/527.4 kB ? eta -:--:--
     ---------------------------------------- 527.4/527.4 kB ? eta 0:00:00

   ---------- ----------------------------- 1/4 [torch_scatter]
   ---------- ----------------------------- 1/4 [torch_scatter]
   -------------------- ------------------- 2/4 [torch_sparse]
   -------------------- ------------------- 2/4 [torch_sparse]
   -------------------- ------------------- 2/4 [torch_sparse]
   ---------------


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pickle
import numpy as np
from rdkit import Chem
from rdkit.Chem import Draw, AllChem
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
import matplotlib.pyplot as plt

In [7]:
dir_add = 'ass2_data/'

with open(dir_add+'pos_data.pkl', 'rb') as f:
    pos_data = pickle.load(f)

with open(dir_add+'type_data.pkl', 'rb') as f:
    type_data = pickle.load(f)

with open(dir_add+'smiles.pkl', 'rb') as f:
    smiles_data = pickle.load(f)

data_split = np.load(dir_add+'data_split.npz')

train_idxes = data_split['train_idx']
test_idxes = data_split['test_idx']

formation_energy = np.load(dir_add+'formation_energy.npz')

fe = formation_energy['y'] # normalized formation energy
mu = formation_energy['mu']
std = formation_energy['sigma']

In [ ]:
# shapes of lists
print("Length of data")
print(f"pos_data: {len(pos_data)}, type_data: {len(type_data)}, smiles: {len(smiles_data)}")
print("Idxes")
print(f"train: {len(train_idxes)}, test: {len(test_idxes)}, sum: {len(train_idxes) + len(test_idxes)}")

In [ ]:
def at_number_to_atom_name(at_number):
    if at_number == 6:
        return 'C'
    elif at_number == 1:
        return 'H'
    elif at_number == 7:
        return 'N'
    elif at_number == 8:
        return 'O'
    elif at_number == 9:
        return 'F'
    elif at_number == 16:
        return 'S'
    else:
        return 'Unknown'

def inspect_structure(idx):
    smile = smiles_data[idx]
    pos = pos_data[idx]
    typ = type_data[idx]

    header = f"{'Atom':^5}│{'Number':^6}│{'x':^10}│{'y':^10}│{'z':^10}"
    line   = "─────┼──────┼──────────┼──────────┼──────────"
    print(header)
    print(line)

    for atom_num, (x, y, z) in zip(typ, pos):
        atom_sym = at_number_to_atom_name(atom_num)
        print(f"{atom_sym:^5}│{atom_num:^6}│{x:>10.3f}│{y:>10.3f}│{z:>10.3f}")
    print("")
    print("")
    print(f'SMILE: {smile}')
    print("")
    print("")
    print(f'Formation Energy: {fe[idx]*std + mu:.3f}')
    print(f'Formation Energy (normalized): {fe[idx]:.5f}')
    mol = Chem.MolFromSmiles(smile)
    if mol:
        # RDKit prefers 2‑D coordinates for nice depictions
        Chem.AllChem.Compute2DCoords(mol)
        img = Draw.MolToImage(mol, size=(300, 300))

        # Display with matplotlib (works both in notebooks and scripts)
        plt.figure(figsize=(3, 3))
        plt.axis('off')
        plt.imshow(img)
        plt.show()

In [ ]:
# methane
# Note how methane has a relatively high formation energy (compared to QM9)
# This correlates with lower thermodynamic stability and higher reactivity
# For example, methane readily burns in oxygen (CH₄ + 2O₂ → CO₂ + 2H₂O)
inspect_structure(0)

In [ ]:
# random structure
inspect_structure(np.random.choice(range(len(smiles_data))))

## Task 1

## Task 2

## Task 3

In [ ]:
def is_valid_smiles(smiles):
    if smiles is None:
        return False
    try:
        mol = Chem.MolFromSmiles(smiles)
        return mol is not None
    except:
        return False

def canonicalize(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            return Chem.MolToSmiles(mol, canonical=True)
        return 'None'
    except:
        return 'None'

In [ ]:
canonicalize("COO"), canonicalize("O(C)O")

In [ ]:
is_valid_smiles("COO"), is_valid_smiles("O(C)O"), is_valid_smiles("C##")